# WebGL backend demo: molecular viewer in the notebook

**WARNING: the WebGL backend requires IPython 3.0 (or the master branch of IPython until 3.0 is released)**

In [ ]:
import numpy as np

from vispy import gloo
from vispy import app
from vispy.util.transforms import perspective, translate, rotate
from vispy.io import load_data_file

app.use_app('ipynb_webgl')

In [ ]:
vertex = """
uniform mat4 u_model;
uniform mat4 u_view;
uniform mat4 u_projection;
uniform vec3 u_light_position;
uniform vec3 u_light_spec_position;

attribute vec3  a_position;
attribute vec3  a_color;
attribute float a_radius;

varying vec3  v_color;
varying vec4  v_eye_position;
varying float v_radius;
varying vec3  v_light_direction;

void main (void) {
    v_radius = a_radius;
    v_color = a_color;

    v_eye_position = u_view * u_model * vec4(a_position,1.0);
    v_light_direction = normalize(u_light_position);
    float dist = length(v_eye_position.xyz);

    gl_Position = u_projection * v_eye_position;

    // stackoverflow.com/questions/8608844/...
    //  ... resizing-point-sprites-based-on-distance-from-the-camera
    vec4  proj_corner = u_projection * vec4(a_radius, a_radius, v_eye_position.z, v_eye_position.w);  // # noqa
    gl_PointSize = 512.0 * proj_corner.x / proj_corner.w;
}
"""

fragment = """
uniform mat4 u_model;
uniform mat4 u_view;
uniform mat4 u_projection;
uniform vec3 u_light_position;
uniform vec3 u_light_spec_position;

varying vec3  v_color;
varying vec4  v_eye_position;
varying float v_radius;
varying vec3  v_light_direction;
void main()
{
    // r^2 = (x - x0)^2 + (y - y0)^2 + (z - z0)^2
    vec2 texcoord = gl_PointCoord* 2.0 - vec2(1.0);
    float x = texcoord.x;
    float y = texcoord.y;
    float d = 1.0 - x*x - y*y;
    if (d <= 0.0)
        discard;

    float z = sqrt(d);
    vec4 pos = v_eye_position;
    pos.z += v_radius*z;
    vec3 pos2 = pos.xyz;
    pos = u_projection * pos;
    //gl_FragDepth = 0.5*(pos.z / pos.w)+0.5;
    vec3 normal = vec3(x,y,z);
    float diffuse = clamp(dot(normal, v_light_direction), 0.0, 1.0);

    // Specular lighting.
    vec3 M = pos2.xyz;
    vec3 O = v_eye_position.xyz;
    vec3 L = u_light_spec_position;
    vec3 K = normalize(normalize(L - M) + normalize(O - M));
    // WARNING: abs() is necessary, otherwise weird bugs may appear with some
    // GPU drivers...
    float specular = clamp(pow(abs(dot(normal, K)), 40.), 0.0, 1.0);
    vec3 v_light = vec3(1., 1., 1.);
    gl_FragColor.rgb = (.15*v_color + .55*diffuse * v_color
                        + .35*specular * v_light);
}
"""

In [ ]:
c = app.Canvas()

@c.connect
def on_initialize(e):
    c.size = 800, 600

    c.program = gloo.Program(vertex, fragment)
    c.view = np.eye(4, dtype=np.float32)
    c.model = np.eye(4, dtype=np.float32)
    c.projection = np.eye(4, dtype=np.float32)
    c.translate = 40
    translate(c.view, 0, 0, -c.translate)

    fname = load_data_file('molecular_viewer/micelle.npz')
    load_molecule(fname)
    load_data()

    c.theta = 0
    c.phi = 0

    c.timer = app.Timer('auto', connect=on_timer, start=True)
    gloo.set_state(depth_test=True, clear_color='black')
    
def load_molecule(fname):
    molecule = np.load(fname)['molecule']
    c._nAtoms = molecule.shape[0]

    # The x,y,z values store in one array
    c.coords = molecule[:, :3]

    # The array that will store the color and alpha scale for all the atoms
    c.atomsColours = molecule[:, 3:6]

    # The array that will store the scale for all the atoms.
    c.atomsScales = molecule[:, 6]

def load_data():
    n = c._nAtoms

    data = np.zeros(n, [('a_position', np.float32, 3),
                        ('a_color', np.float32, 3),
                        ('a_radius', np.float32, 1)])

    data['a_position'] = c.coords
    data['a_color'] = c.atomsColours
    data['a_radius'] = c.atomsScales

    c.program.bind(gloo.VertexBuffer(data))

    c.program['u_model'] = c.model
    c.program['u_view'] = c.view
    c.program['u_light_position'] = 0., 0., 2.
    c.program['u_light_spec_position'] = -5., 5., -5.

@c.connect
def on_key_press(event):
    if event.text == ' ':
        if c.timer.running:
            c.timer.stop()
        else:
            c.timer.start()

def on_timer(event):
    c.theta += .25
    c.phi += .25
    c.model = np.eye(4, dtype=np.float32)

    rotate(c.model, c.theta, 0, 0, 1)
    rotate(c.model, c.phi, 0, 1, 0)

    c.program['u_model'] = c.model
    c.update()

@c.connect
def on_resize(event):
    width, height = event.size
    gloo.set_viewport(0, 0, width, height)
    c.projection = perspective(25.0, width / float(height), 2.0, 100.0)
    c.program['u_projection'] = c.projection

@c.connect
def on_mouse_wheel(event):
    c.translate -= event.delta[1]
    c.translate = max(-1, c.translate)
    c.view = np.eye(4, dtype=np.float32)

    translate(c.view, 0, 0, -c.translate)

    c.program['u_view'] = c.view
    c.update()

@c.connect
def on_draw(event):
    gloo.clear()
    c.program.draw('points')
    
c.show()
#or
#from vispy.app.backends.ipython import VispyWidget
#w = VispyWidget()
#w.set_canvas(c)
#w

In [ ]:
c.timer.stop()